In [79]:
import pandas as pd
import os
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import mean_absolute_error
MODEL_PATH = '../03.SavedModels'
MODEL_NAMES = ['baseline_lstm_v7','baseline_lstm_v8']
MODE_ENSEMBLE = 'mean'

In [80]:
oof_list   = []
preds_list = []
train_df = pd.read_csv('../01.Data/train_folds.csv')[['id','pressure','u_out']]
test_df  = pd.read_csv('../01.Data/test.csv')[['id','u_out']]

for name in tqdm(MODEL_NAMES):    
    oof_df  = pd.read_csv(os.path.join(MODEL_PATH,name,'oof_preds.csv')).sort_values(by = ['id'])
    pred_df = pd.read_csv(os.path.join(MODEL_PATH,name,'test_preds.csv')).sort_values(by = ['id']) 
    oof_df = train_df.merge(oof_df,how = 'left',on = ['id']).sort_values(by = ['id']).fillna(-1)
    pred_df = test_df.merge(pred_df,how = 'left',on = ['id']).sort_values(by = ['id']).fillna(-1)
    
    print(f"========== OOF METRIC: {name} ========")
    tmp = oof_df.loc[oof_df['u_out']==0]
    print(mean_absolute_error(tmp.oof,tmp.pressure))
        
    oof_list.append(oof_df.oof.values)
    preds_list.append(pred_df.preds.values)
    del tmp,oof_df,pred_df

========== OOF METRIC: baseline_lstm_v7 ========
0.17184925552389443
========== OOF METRIC: baseline_lstm_v8 ========
0.19504527290441698



In [81]:
if MODE_ENSEMBLE == 'mean':
    oof_ens   = np.mean(np.vstack(oof_list),axis = 0)
    preds_ens = np.mean(np.vstack(preds_list),axis = 0)
elif MODE_ENSEMBLE == 'median':
    oof_ens   = np.median(np.vstack(oof_list),axis = 0)
    preds_ens = np.median(np.vstack(preds_list),axis = 0)
print(f"{'='*5} ENSEMBLE MEAN {'='*5}")
train_df['oof'] = oof_ens
tmp = train_df.loc[train_df['u_out']==0]
print(mean_absolute_error(tmp.oof,tmp.pressure))
test_df['pressure'] = preds_ens
test_df = test_df[['id','pressure']]

===== ENSEMBLE MEAN =====
0.16947476477255635


In [70]:
test_df.to_csv('submission_simple_ensemble.csv',index = False)